In [1]:
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import count

In [2]:
# Building a Spark-Session using a spark-Session API.
# 1 Spark-Session Per JVM.
# Spark Driver Connects with Spark-Executor by creating a Spark Session.
spark = SparkSession.builder.appName("SPARK_01").getOrCreate()

23/10/24 18:52:57 WARN Utils: Your hostname, Sivas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.65.79 instead (on interface en0)
23/10/24 18:52:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/24 18:52:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read the File as a CSV format into the Spark DataFrame by inferring the Schema 
# And specifying that the file has a Header.
df  = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("mnm_dataset.csv")

In [4]:
# Spark consists of 2 major operation for Data Distribution.
# 1. Transformation. -->> It transforms the original Spark DataFrame into New Dataframe with a property of immutability.
# 2. Action.         -->> It mutates the most Optimized Dataframe from the data lineage into a new dataframe.

# Transform Operation
sdf = df.filter(df.Color == "Red")

# Action Operation
sdf.show(5)

+-----+-----+-----+
|State|Color|Count|
+-----+-----+-----+
|   TX|  Red|   20|
|   NV|  Red|   98|
|   CO|  Red|   82|
|   CO|  Red|   12|
|   CO|  Red|   17|
+-----+-----+-----+


In [5]:
# 1. Select State, Color, Count Where Count is greater than 20
mm_df =  df.select("State", "Color", "Count").where(df.Count > 20).show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
|   AZ| Green|   46|
|   NV|   Red|   43|
|   CO|  Blue|   95|
+-----+------+-----+


In [6]:
# 2. Select State, Color, Count  Group by State, Color aggregated by Count.
mm_df_01 = df.select("State", "Color", "Count").groupBy("State", "Color").agg(count("Count").alias("Total")).orderBy("Total", ascending = True)

In [7]:
mm_df_01.show(n=600, truncate=False)

# Write Spark Dataframe to Drive (CSV).
# mm_df_01.write.csv("test_01.csv")

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|WY   |Brown |1532 |
|UT   |Green |1591 |
|WY   |Orange|1595 |
|CA   |Blue  |1603 |
|NV   |Red   |1610 |
|TX   |Blue  |1614 |
|OR   |Yellow|1614 |
|OR   |Brown |1621 |
|CO   |Red   |1624 |
|WA   |Blue  |1625 |
|WY   |Yellow|1626 |
|UT   |Brown |1631 |
|OR   |Green |1634 |
|AZ   |Blue  |1636 |
|NM   |Blue  |1638 |
|TX   |Brown |1641 |
|CO   |Orange|1642 |
|UT   |Yellow|1645 |
|OR   |Red   |1645 |
|OR   |Blue  |1646 |
|AZ   |Red   |1648 |
|TX   |Orange|1652 |
|AZ   |Yellow|1654 |
|UT   |Blue  |1655 |
|CA   |Red   |1656 |
|CO   |Brown |1656 |
|NV   |Brown |1657 |
|CA   |Orange|1657 |
|WA   |Orange|1658 |
|WA   |Yellow|1663 |
|WY   |Blue  |1664 |
|NM   |Orange|1665 |
|WA   |Brown |1669 |
|WY   |Red   |1670 |
|WA   |Red   |1671 |
|NV   |Blue  |1673 |
|NV   |Yellow|1675 |
|AZ   |Green |1676 |
|UT   |Red   |1680 |
|NM   |Green |1682 |
|UT   |Orange|1684 |
|NM   |Brown |1687 |
|NM   |Yellow|1688 |
|AZ   |Orange|1689 |
|NM   |Red   

In [11]:
# 3. Select State, Color, Count where State = CA Group by State, Color aggregated by Count.
mm_df_02 = df.select("State", "Color", "Count").where(df.State == "WA").groupBy("State", "Color").agg(count("Count").alias("Total")).orderBy("Total", ascending = True)


In [9]:
mm_df_02.show(n=60, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|WA   |Blue  |1625 |
|WA   |Orange|1658 |
|WA   |Yellow|1663 |
|WA   |Brown |1669 |
|WA   |Red   |1671 |
|WA   |Green |1779 |
+-----+------+-----+


In [12]:
import sys
from pyspark.sql import SparkSession as sps
from pyspark.sql.functions import count

table_name = 'default.loan_risks_upload'
source_data = 'dbfs:/FileStore/mnm_dataset.csv'
source_format = 'CSV'

spark = sps.builder.appName("DATABRICKS_MMCOUNT_SPARK_JOB").getOrCreate()

spark.sql()
# Read the File as a CSV into Spark Dataframe by inferring Schema.
# And specifying that the file has Header.
#
# Example: Run spark_job in DataBricks LakeHouse.
# Upload the mnm_dataset.csv in the Databricks DBFS.
# Copy the URL of the File in the DBFS in the Read Function.
df = spark.read.format("csv").option("header", "true").option("mergeSchema", "true").option("inferSchema", "true").load(source_data)



df.sql_ctx.sql("DROP TABLE IF EXISTS " + table_name)

df.sql_ctx.sql("CREATE TABLE " + table_name + " (" \
                                              "state STRING, " + \
               "color STRING, " + \
               "count STRING)"
               )

df.sql_ctx.sql("COPY INTO " + table_name + \
               " FROM '" + source_data + "'" + \
               " FILEFORMAT = " + source_format
               )

loan_risks_upload_data = df.sql_ctx.sql("SELECT * FROM " + table_name)

display(loan_risks_upload_data)

23/10/24 18:56:10 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: [NOT_SUPPORTED_COMMAND_WITHOUT_HIVE_SUPPORT] CREATE Hive TABLE (AS SELECT) is not supported, if you want to enable it, please set "spark.sql.catalogImplementation" to "hive".;
'CreateTable `spark_catalog`.`default`.`loan_risks_upload`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists
